In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_excel("../data/sods_limpo.xlsx")

In [3]:
df.head(5)

,idade,genero,etnia,pcd,vive_no_brasil,estado_moradia,nivel_ensino,formacao,tempo_experiencia_dados,linguagens_preferidas,bancos_de_dados,cloud_preferida,cargo
0,39,Masculino,Parda,Não,True,Distrito Federal (DF),Pós-graduação,Computação / Engenharia de Software / Sistemas...,de 1 a 2 anos,r,"postgresql, oracle, mysql, sql server",Amazon Web Services (AWS),DBA/Administrador de Banco de Dados
1,53,Masculino,Branca,Não,True,Distrito Federal (DF),Pós-graduação,Computação / Engenharia de Software / Sistemas...,de 3 a 4 anos,python,"postgresql, mysql, oracle, db2",Amazon Web Services (AWS),Desenvolvedor/ Engenheiro de Software/ Analist...
2,27,Masculino,Branca,Não,True,Minas Gerais (MG),Doutorado ou Phd,Estatística/ Matemática / Matemática Computaci...,de 4 a 6 anos,python,google bigquery,Não sei opinar,Cientista de Dados/Data Scientist
3,46,Feminino,Branca,Não,True,Pará (PA),Pós-graduação,Computação / Engenharia de Software / Sistemas...,Não tenho experiência na área de dados,python,microsoft access,Amazon Web Services (AWS),Desenvolvedor/ Engenheiro de Software/ Analist...
4,32,Feminino,Parda,Não,True,Ceará (CE),Pós-graduação,Ciências Biológicas/ Farmácia/ Medicina/ Área ...,Não tenho experiência na área de dados,python,google bigquery,Google Cloud (GCP),Professor


## Extraindo Informações Sobre As Variaveis Categoricas

In [4]:
df.select_dtypes(include='object').describe()

,genero,etnia,pcd,estado_moradia,nivel_ensino,formacao,tempo_experiencia_dados,linguagens_preferidas,bancos_de_dados,cloud_preferida,cargo
count,2981,2981,2981,2981,2981,2981,2981,2981,2981,2981,2981
unique,3,7,3,23,7,10,7,21,1429,5,18
top,Masculino,Branca,Não,São Paulo (SP),Graduação/Bacharelado,Computação / Engenharia de Software / Sistemas...,de 1 a 2 anos,python,google bigquery,Amazon Web Services (AWS),Analista de Dados/Data Analyst
freq,2212,1917,2927,1220,1143,1102,906,2581,227,1153,639


In [5]:
df["bancos_de_dados"].unique()

array(['postgresql, oracle, mysql, sql server',
       'postgresql, mysql, oracle, db2', 'google bigquery', ...,
       'google bigquery, postgresql, mysql',
       'postgresql, oracle, mariadb, hbase, mysql',
       'oracle, firebird, sql server'], shape=(1429,), dtype=object)

## Variáveis Categoricas do dataframe 

    - nivel_ensino (ordinal) == oneHotEncoder 
    - tempo_de_experiencia (ordinal) == oneHotEncoder
    - genero (nominal) == oneHotEnconder 
    - etnia (nominal) == OneHotEnconder 
    - estado_moradia (nominal) == OnehotEnconder
    - pcd e vive_no_brasil (binaria) = mapeamento para 0 || 1
    
    - linguagem_preferidas e bancos_de_dados (OneHotEncoder com CountVectorizer)

    - cargo (Vai ser meu target)
    

Analisando os valores unicos da coluna "estado_moradia", pois é uma das colunas com mais valores unicos e mapeando a fim de diminuir o numero de novas colunas criadas com o OneHotEnconder

## Realizando o Mapeamento Binário

In [6]:
# Mapeia 'Sim' para 1 e 'Não' para 0
df['pcd'] = df['pcd'].map({'Sim': 1, 'Não': 0})

# Converte True para 1 e False para 0
df['vive_no_brasil'] = df['vive_no_brasil'].astype(int)

## Realizando o OneHotEncoder

In [7]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

colunas_OneHot = ['genero','etnia','formacao','cloud_preferida','estado_moradia','nivel_ensino','tempo_experiencia_dados']

encoded_data = encoder.fit_transform(df[colunas_OneHot])

new_column_names = encoder.get_feature_names_out(colunas_OneHot)

encoded_df = pd.DataFrame(encoded_data, columns=new_column_names, index=df.index)

df = pd.concat([df.drop(colunas_OneHot, axis=1), encoded_df], axis=1)


## Realizando o OneHotEnconde usando o CountVectorizer

descrição: utiliza-se o CountVectorizer para realizar o OneHot quando se tem texto e é necessário quebra-lo pra forma novas colunas individuais

In [8]:
df['linguagens_preferidas'] = df['linguagens_preferidas'].fillna('')

#Inicializar o CountVectorizer com o token_pattern CORRIGIDO
#O padrão r'(?u)\b\w+\b' captura "palavras" de 1 ou mais caracteres, incluindo 'r'
vectorizer = CountVectorizer(
    binary=True,
    token_pattern=r'(?u)\b\w+\b'
)

#Aplicar o vectorizer à coluna para aprender o vocabulário e transformar os dados
linguagens_vetorizadas = vectorizer.fit_transform(df['linguagens_preferidas'])

#Criar um novo DataFrame com as colunas para cada linguagem
linguagens_df = pd.DataFrame(
    linguagens_vetorizadas.toarray(),
    columns=vectorizer.get_feature_names_out(),
    index=df.index
)

df = pd.concat([df.drop('linguagens_preferidas', axis=1), linguagens_df], axis=1)


In [9]:
df['bancos_de_dados'] = df['bancos_de_dados'].fillna('')

sinonimos = {
    # Bancos relacionais
    "mysql": ["mysql", "my sql", "my-sql", "my_sql", "pwa microsoft - mysql ou sql"],
    "postgresql": ["postgres", "postgresql", "postgre"],
    "sqlserver": ["sql server", "sqlserver", "ms sql", "mssql", "sql dbx"],
    "oracle": ["oracle", "oracledb", "autonomous db", "sap hana", "hana"],
    "db2": ["db2"],
    "sqlite": ["sqlite", "sq lite"],
    "access": ["access", "microsoft access"],
    "mariadb": ["mariadb"],
    "firebird": ["firebird"],
    "informix": ["informix", "ibm informix"],

    # NoSQL
    "mongodb": ["mongodb", "mongo db", "mongo"],
    "cassandra": ["cassandra"],
    "document_stores": ["cosmos db", "couchdb", "cruxdb", "datomic", "dynamodb", "firebase"],
    "graph_db": ["neo4j"],
    "time_series": ["influxdb"],
    "key_value": ["redis"],
    "search_engines": ["elasticsearch", "solr"],

    # Cloud AWS
    "aws": ["amazon aurora", "amazon aurora ou rds", "aurora",
            "amazon redshift", "redshift",
            "amazon athena", "rds", "s3"],

    # Cloud GCP
    "gcp": ["bigquery", "google bigquery", "google big query",
            "big table", "bigtable",
            "cloud spanner",
            "google analytics",
            "google cloud storage", "gcs",
            "google firestore",
            "google sheets",
            "google"],

    # Cloud Azure
    "azure": ["azure", "ms azure",
              "azure sql",
              "azure synapse", "synapse",
              "azure blob storage", "azue blob storage",
              "azure data lake", "azure adls",
              "azure data explorer",
              "azure storage account"],

    # Plataformas/Lakes/ETL
    "data_lakes": ["databricks", "databricks sql",
                   "deltalake", "denodo", "dremio", "dali - hpcc system", "hpcc systems"],
    "bi_tools": ["cognos", "sap bo", "microstrategy", "powerbi", "superset",
                 "tableau", "sas", "sas libs", "spss", "splunk", "metabase"],
    "dev_tools": ["dbeaver", "knime", "kobotoolbox", "espaider"],

    # Arquivos / Formatos
    "arquivos": [".csv", "bases excel e csv extraídas direto no site", "excel",
                 "nosso banco é no excel", "odbc", "office"],

    # Dados internos / externos
    "dados_internos": [
        "dados internos", "dados não estruturados", "base interna",
        "base de dados local", "dados alternativos e dados internos da empresa",
        "banco de dados próprio da empresa", "fontes internas e de clientes",
        "fontes da empresa em html ou csv", "interno"
    ],
    "dados_publicos": ["dados públicos externos", "datasus", "receita"],
    "dados_geoespaciais": ["dados geoespaciais"],
    "web_apis": ["api", "consumo de apis com azure functions", "webscraping"],
    "midias_digitais": ["diversas fontes de marketing digital", "midias sociais", "rd station", "salesforce", "omie", "crm", "sistema de crm"],

    # SAP
    "sap": ["sap", "sap ecc", "sap business", "sap hana"],

    # Big Data / Processamento
    "big_data": ["hadoop", "hive", "hbase", "impala", "presto", "vertica", "teradata", "snowflake", "splunk", "hue", "ibm blue mix"],

    # Outros
    "lista_sharepoint": ["lista do sharepoint", "sharepoint"],
    "pesquisa_empirica": ["pesquisas empírica que eu coleto os dados"],
    "prefiro_nao_informar": ["prefiro não informar"],
    "nao_uso": ["não uso"],
    "outro": ["outro", "solução proprietária", "ooo", "nsa", "sintax"],
    "python": ["python"],
}



# Função para normalizar
def normalizar_nome(nome):
    nome = nome.strip().lower()
    for padrao, variantes in sinonimos.items():
        if nome in variantes:
            return padrao
    return nome  # se não bater, mantém

# Quebrar por vírgula, limpar e normalizar
def preprocessar_bancos(texto):
    itens = [item.strip() for item in texto.split(',')]
    return ",".join([normalizar_nome(item) for item in itens if item])

df['bancos_de_dados'] = df['bancos_de_dados'].apply(preprocessar_bancos)

# Tokenizer customizado
custom_tokenizer = lambda text: [item.strip() for item in text.split(',')]

# Vectorizer
vectorizer_db = CountVectorizer(
    binary=True,
    tokenizer=custom_tokenizer
)

db_vetorizados = vectorizer_db.fit_transform(df['bancos_de_dados'])

bancos_df = pd.DataFrame(
    db_vetorizados.toarray(),
    columns=vectorizer_db.get_feature_names_out(),
    index=df.index
)

# Concatenar de volta
df = pd.concat([df.drop('bancos_de_dados', axis=1), bancos_df], axis=1)


c:\Users\morae\workspace\jornada\censo_preditivo\venv_novo\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## Normalizando Dados e comparativo estatistico da coluna idade antes e depois da normalização 

In [10]:
idade_antes_normalizada = df['idade'].describe()

In [11]:
colunas_para_normalizar = ['idade']

scaler = MinMaxScaler()

df[colunas_para_normalizar] = scaler.fit_transform(df[colunas_para_normalizar])

In [12]:
idade_depois_normalizada = df['idade'].describe()

In [13]:
matriz_comparativa = pd.DataFrame({
    'Antes da Normalizacao (Idade em anos)': idade_antes_normalizada,
    'Depois da Normalizacao (0 a 1)': idade_depois_normalizada
})

matriz_comparativa

,Antes da Normalizacao (Idade em anos),Depois da Normalizacao (0 a 1)
count,2981.000000,2981.000000
mean,30.334452,0.342624
std,6.439328,0.178870
min,18.000000,0.000000
25%,26.000000,0.222222
50%,29.000000,0.305556
75%,34.000000,0.444444
max,54.000000,1.000000


In [14]:
df.to_excel('../data/sods_processado.xlsx', index=False)